Another potentially useful connection from the work plan species are to the terrestrial vertebrates that have been modeled and mapped under the Gap Analysis Project. GAP species include mapped range and habitats (ecological systems classification of the GAP Land Cover 2001 product) along with a set of habitat relationship parameters used in the modeling process, associated scientific references, and other details that may prove useful in working with the species status assessment process.

In [1]:
import requests
import bispy
import json
from IPython.display import display
from joblib import Parallel, delayed
import jsonschema

gap = bispy.gap.Gap()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
# Open up the cached workplan species
with open("../cache/workplan_species.json", "r") as f:
    workplan_species = json.loads(f.read())

In [3]:
# Gap only models mammals, birds, reptiles, and amphibians, so restrict the list to just those guilds
possible_gap_species = [s for s in workplan_species if s["Guild"] in ["Birds","Mammals","Reptiles","Amphibians"]]
name_list = helperfunctions.name_list(possible_gap_species)

In [4]:
%%time
# Use joblib to run multiple requests for records in parallel via scientific names
gap_results = Parallel(n_jobs=8)(delayed(gap.gap_species_search)(name, name_source) for name, name_source in name_list)


CPU times: user 293 ms, sys: 74.6 ms, total: 368 ms
Wall time: 1min 35s


In [7]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("../cache/gap.json", gap_results))

{'Doc Cache File': 'cache/gap.json',
 'Number of Documents in Cache': 95,
 'Document Number 17': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-07-26T19:02:59.039926',
   'status_message': 'Exact Match',
   'api': "https://www.sciencebase.gov/catalog/items?parentId=527d0a83e4b0850ea0518326&format=json&fields=identifiers,files,webLinks,distributionLinks,dates&filter=itemIdentifier%3D{'key': 'Tamias minimus atristriatus'}"},
  'parameters': {'Scientific Name': 'Tamias minimus atristriatus',
   'Name Source': 'Lookup Name'},
  'GAP Species': {'GAP Habitat Map Item': 'https://www.sciencebase.gov/catalog/item/58fa6cb7e4b0b7ea54525935',
   'GAP Range Map Item': 'https://www.sciencebase.gov/catalog/item/59f5e2a3e4b063d5d307dfc3',
   'GAP Habitat Map WMS': 'https://www.sciencebase.gov/geoserver/CONUS_HabMap_2001/wms?service=WMS&version=1.1.0&request=GetCapabilities',
   'GAP Modeling Database Parameters URL': 'https://www.sciencebase.gov/catalog/file/get/58fa6cb7e4b0b

# Schema Validation
The GAP species collection generated here is the most complex of the lot in terms of the number and complexity of properties in the structure. The function process here backed by the functions in the bispy package assembled the full slate of information put together in ScienceBase for the official release of the 1719 species and subspecies released from the 2001 era modeling process into a single set of records. More robust documentation is available in the [ScienceBase collection](https://www.sciencebase.gov/catalog/item/527d0a83e4b0850ea0518326) than what is captured in the schema document.

In [8]:
gap_schema = helperfunctions.load_schema('gap')
display(gap_schema)

jsonschema.validate(gap_results, gap_schema)

{'definitions': {'items': {'$id': '#items',
   'type': ['object', 'array'],
   'title': 'Generic container for items in a dataset',
   'description': 'A JSON array or object property containing one or more items in a dataset or data structure within a dataset.'}},
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'http://data.usgs.gov/property_registry/',
 'type': 'array',
 'title': 'GAP Species Summary Collection',
 'description': 'A dataset containing the results of a search for Gap Analysis Project Species in ScienceBase. The process assembles several collections of details about the species for further analysis and other uses.',
 'items': {'$ref': '#/definitions/items',
  'properties': {'processing_metadata': {'$ref': 'common_properties.json#/definitions/processing_metadata'},
   'parameters': {'$ref': 'common_properties.json#/definitions/parameters',
    'required': ['Scientific Name', 'Name Source'],
    'properties': {'Scientific Name': {'$ref': 'common_properties.j